# The DBT Workflow - Redshift

### Introduction

Now that we have gotten our DBT account connected to our data warehouse and our github repository, it's time to see how we can use DBT with our data warehouse.  Now remember, that there are a few main benefits of using DBT:


1. It provides an opinionated workflow for writing our queries
2. It provides opinionated *file structure* for organizing our SQL queries
1. It allows to quickly turn SQL SELECT statements into new SQL tables populated with that selected data


In this lesson, we'll focus on the two benefits: we'll see how DBT can allow us to quickly create and populate new tables, and we'll see how it enforces a proper development workflow.

### Querying our Database in Redshift

Ok, so before moving to DBT, let's explore some of the data we have directly in Redshift.  When we initially created the redshift database, AWS should have inserted some sample data for us.

So now, once logged into AWS, go to the Redshift dashboard, and from there, click on `query_editor`.

> <img src="./query_editor.png" width="30%">

The redshift query editor can connect us to any of our redshift clusters and issue commands on our database -- kind of like our `psql` script.  So next, we need to connect our query editor to a specific cluster and database.  To do this,  click on `Change connection` over to the right towards the top of the screen.

<img src="./change_conn.png" width="60%">

And as you can see below, from there we can create a new connection, and specify the cluster, database name and database user to connect to the proper redshift cluster.

> <img src="./redshift_query.png" width="50%">

And now we can see some of the tables and sample data that AWS already loaded into our redshift cluster.

> <img src="./tables_avail.png" width="40%">

Focusing in on the users table above, we can see that the table has a lot of available columns.  Let's just do an initial query in our redshift query editor where we select a subset of the columns from our `users` table, and then we perform a similar query in DBT. 

```SQL
SELECT userid, username, firstname, city, state, email, phone FROM users LIMIT 20;
```

> <img src="./run_query.png" width="60%">

And upon clicking on the orange `run` button, we see the results below.

<img src="./results.png" width="60%">

### Moving to DBT

Ok, now let's try to run this same query from DBT.  First, remember that we should have already connected DBT to our redshift database.  In fact, let's see this.  Once your logged into DBT, click on the sidebar, and then click on Account Settings. 

> <img src="./account_settings.png" width="30%">

We can then see the connection to redshift from our analytics project.

> <img src="./account_redshift.png" width="80%">

So click on `Connection: Redshift`, and we can see the details of that connection.  

> <img src="./redshift_conn.png" width="50%">

### DBT with Git

Ok, so now it's about time to query our from DBT.  So click on the `Analytics` dropdown at the top, followed by the `Analytics` tab in the dropdown.

> <img src="./analytics_project.png" width="60%">

And from there return to the codebase by clicking on the green `Start Developing` button.  

Now, when we return to the codebase, we start off on the master branch in git.  But DBT *will not allow us* to make any changes or add new code directly on the master branch.  

> <img src="./create_branch.png" width="60%">

We can see that by looking underneath the green button.  It tells us the current branch is master, and specifies that this branch is read only, `branch: master (read-only)`.

> **Thanks DBT!** Here, DBT is enforcing a proper a proper git workflow.  Developers should never make changes directly on master, because that code will affect the codebase that the rest of the team is working on.  If we made a mistake or broke some tests and it made it to master, the rest of the development team would suffer.  So instead the workflow is to (1) create a new branch (2) make changes on that branch and then (3) eventually merge that code onto master by opening a pull request.

So instead, we need to first click on the `Create a new branch` button.  And then, we can name the branch something related to the work we are about to complete.

> <img src="./dim_customers_model.png" width="60%"> 

Once we click submit, DBT will automatically create the `build_dim_customers_model` branch, and also checkout that branch for us.   Notice that now the green button has changed to `open pull request`, and that underneath it tells us our current branch `branch: build_dim_customers_model`.

> <img src="./resulting_branch.png" width="60%">

### Performing the query

And from once we have created this new branch, from there we just need to create a new folder called `models` (not in the `example` folder), and then under that folder a file called `dim_customers.sql`.  Once this new file is created, we can write a query similar to what we wrote in redshift.

> <img src="./preview_data_sql.png" width="80%">

```sql
SELECT userid, username, firstname, lastname, city, state, email, phone FROM users
```

> Notice that there is no semi-colon at the end of our SQL statement. DBT will add that for us.  

And from there, we can can query the database by clicking on the teal `preview data` button towards the bottom.

### Creating a new table

Ok, now currently DBT just queried our data displayed to us the results.  But remember, that one purpose of DBT is to turn those results into a newly populated table.  If you look at the *very* bottom of the screen you'll see a field for `Runs`.  

> This is essentially the equivalent of entering a command into the terminal.

Then if we call the `dbt run` command, we'll see that the result of our query will also create and insert data into a new table.

>  <img src="./runs.png" width="100%">

```bash
dbt run --models dim_customers
```

> Above we specify to only run the code in `dim_customers.sql`, and to not run *all* of the code in our codebase. 

And then from there, will be taken to the Runs dashboard.

> <img src="./built_model.png" width="100%">

Now just from our `SELECT` statement, DBT supposedly created and inserted in some data.  We can see this if we click on `dim_customers`, and then move the green radio button from `Summary` over to `Details`.

> <img src="./dim_customers.png" width="100%">

Then scroll down.

> <img src="./compiled_dbt.png" width="100%">

So here, we can see that DBT turned our `SELECT` statement into some SQL that creates a new table `dev.dbt_jkatz.dim_customers_dbt_tmp`, and inserts in the data selected from our query.  

> Unpacking the name of our table, `dev` is the name of the ddatabase, `dbt_jkatz` is the newly created schema, and `dim_customers_dbt_tmp` is the name of the newly created table -- generated from the file name of our model, `dim_customers.sql`.    

### Returning to Redshift

So at this point we've used DBT to quickly create a new table.  Let's also confirm that it has in fact changed our data warehouse by returning to redshift.  If we return to the query editor in redshift, we can see that we have a new schema, `dbt_jkatz` (yours will be different).

<img src="./new_schema.png" width="30%">

And then if we take a look at the tables in that schema, we can see that there is a table in that schema, with our queried data, and the appropriate columns.

> If we want, we an view some of that data with a call to `SELECT * FROM dbt_jkatz.dim_customers LIMIT 20;`.

<img src="./queried_data.png" width="80%">

### Updating the codebase

Alright, so now that we've confirmed that our development database was updated properly with our DBT code, the next step is to commit our changes to the codebase, and perhaps merge our changes to master.

So click on the green commit button.

> <img src="./make_commit.png" width="80%">

And add a commit message like `added dim_customers model`.

> <img src="./dim_customers_model_commit.png" width="80%">

Then, click on `open pull request`.

<img src="./pull_request.png" width="80%"> 

<img src="./opened_pull.png" width="80%">

Then click create pull request, and add a commit message, then click on `Confirm merge`.

<img src="./confirm_merge.png" width="60%">

Finally, click on `Merge pull request` to merge our updated code with the master branch.

<img src="./merge_pull.png" width="60%">

If we go to the master branch, we will find our `models/dim_customers.sql` file with the code from DBT.

<img src="./updated_master.png" width="60%">

And our work is complete :)

### Summary

In this lesson, we saw the workflow of DBT.  DBT connects to both our data warehouse and to our Github accounts.  Once these two components are connected, we then use DBT to turn our SELECT query into a table in our data warehouse.  We performed this by first performing the query to see what it selected, and then creating a new table in our data warehouse with the command:

`dbt run --models dim_customers`

We and we confirmed the change was made by querying redshift directly.

Now at this point, we still have not added our changes to the DBT codebase on the DBT master branch.  So to do this, we used DBT to create a new commit first on our branch, and then merged those changes onto master.